# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import seaborn as sb
import json
import time
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.autograd import Variable

from collections import OrderedDict
import PIL
from workspace_utils import active_session

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
# Normalization means and standard deviations
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

In [3]:
# DONE: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation((45)),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.ToTensor(),
                                       transforms.Normalize(norm_mean, norm_std)])

valid_transforms = transforms.Compose([transforms.Resize(225),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(norm_mean, norm_std)])

test_transforms = transforms.Compose([transforms.Resize(225),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(norm_mean, norm_std)])

# DONE: Load the datasets with ImageFolder
train_data = datasets.ImageFolder('flowers/train', transform=train_transforms) 
valid_data = datasets.ImageFolder('flowers/valid', transform=valid_transforms)
test_data = datasets.ImageFolder('flowers/test', transform=test_transforms)

# DONE: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 64, shuffle = True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size = 64, shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size = 64, shuffle = True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [9]:
# Define the pretrained model to use
model = models.resnet152(pretrained = True)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [10]:
# Use CUDA GPU accleration if it's available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Block backpropogation
for param in model.parameters():
    param.requires_grad = False

# Define the classifier
classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(2048, 1024)),
                                        ('relu', nn.ReLU()),
                                        ('drop1', nn.Dropout(p=0.25)),
                                        ('fc2', nn.Linear(1024, 512)),
                                        ('relu', nn.ReLU()),
                                        ('drop2', nn.Dropout(p=0.25)),
                                        ('fc3', nn.Linear(512, 256)),
                                        ('relu', nn.ReLU()),
                                        ('drop3', nn.Dropout(p=0.25)),
                                        ('fc4', nn.Linear(256, 102)),
                                        ('output', nn.LogSoftmax(dim=1))]))

# Define the model 
model.fc = classifier
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr = .0005)

model.to(device);

In [11]:
## DONE: Use a pretrained model to classify the flower images
start_time = time.time()
    
epochs = 6
steps = 0
print_every = 5
running_loss = 0

train_losses, valid_losses = [], []
valid_accuracy = []
    
for epoch in range(epochs):
    for images, labels in trainloader:
        steps += 1
        
        # move input and label tensor to the default device
        images, labels = images.to(device), labels.to(device)
            
        optimizer.zero_grad()
        
        log_ps = model.forward(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
            
        # Validation on the seperate validater set
        if steps % print_every == 0:
            model.eval()
            valid_loss = 0
            accuracy = 0
            
            for images, labels in validloader:
                
                images, labels = images.to(device), labels.to(device)
                    
                log_ps = model(images)
                batch_loss = criterion(log_ps, labels)
                valid_loss += batch_loss.item()
                    
                # Calculate accuracy
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                    
            train_losses.append(running_loss/len(trainloader))
            valid_losses.append(valid_loss/len(validloader))
            valid_accuracy.append(accuracy/len(validloader))
                
            print("Epoch: {}/{}..".format(epoch+1, epochs),
                  "Training Loss: {:.3f}".format(running_loss/len(trainloader)),
                  "Validation Loss: {:.3f}".format(running_loss/len(validloader)),
                  "Validation Accuracy: {:.3f}".format(accuracy/len(validloader)))
            running_loss = 0
            model.train()
                
end_time = time.time()
total_time = time.strftime('%H:%M:%S', time.gmtime(start_time - end_time))

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
print(total_time)

plt.plot(train_losses, label = 'Training Loss')
plt.plot(valid_losses, label = 'Validation Loss')
plt.legend(frameon='False');

In [ ]:
plt.plot(valid_accuracy, label = 'Validation Accuracy');

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# DONE: Do validation on the test set
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
model.to(device)
    
epochs = 5
steps = 0    
print_every = 5
test_losses, test_accuracy = [], []
        
for epoch in range(epochs):
    for images, labels in testloader:
        steps += 1
        images, labels = images.to(device), labels.to(device)
            
        if steps % print_every == 0:
            model.eval()
            test_loss = 0
            accuracy = 0
                
            for images, labels in testloader:
                        
                # move input and label tensor to the default device
                images, labels = images.to(device), labels.to(device)
                    
                log_ps = model(images)
                test_batch_loss = criterion(log_ps, labels)
                test_loss += test_batch_loss.item()

                # Calculate accuracy
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                        
            test_losses.append(test_loss/len(testloader))
            test_accuracy.append(accuracy/len(testloader))
                        
            print(("Epoch: {}/{}..".format(epoch+1, epochs)),
                  ("Test Loss: {:.3f}..".format(test_loss/len(testloader))),
                  ("Test Accuracy: {:.3f}".format(accuracy/len(testloader))))
                
            test_loss = 0

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
classifier

In [ ]:
optimizer.state_dict

In [ ]:
optimizer.param_groups

In [ ]:
model

In [ ]:
model.state_dict

In [ ]:
model.fc.weight

In [ ]:
model.fc.bias

In [ ]:
# DONE: Save the checkpoint 
model.class_to_idx = train_data.class_to_idx

checkpoint = {'arch': model,
              'norm_mean': norm_mean,
              'norm_std': norm_std,
              'criterion': criterion,
              'classifier': classifier,
              'optimizer': optim.Adam(model.fc.parameters(), lr = .0005),              
              'class_to_idx': model.class_to_idx,
              'model_state_dict': model.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [3]:
# Define a function to load the checkpoint later on
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['arch']
    optimizer = checkpoint['optimizer']
    model.load_state_dict(checkpoint['model_state_dict'])
    model.fc = checkpoint['classifier']
    criterion = checkpoint['criterion']
    model.class_to_idx = checkpoint['class_to_idx']
    norm_mean = checkpoint['norm_mean']
    norm_std = checkpoint['norm_std']
    
    return model, optimizer, criterion, norm_mean, norm_std

In [9]:
# DONE: Write a function that loads a checkpoint and rebuilds the model
model, optimizer, criterion, norm_mean, norm_std = load_checkpoint('checkpoint.pth')

In [ ]:
# Check that everything loaded correctly
print('model:', model, '\n\n')

In [ ]:
model.class_to_idx

In [ ]:
print('optimizer:', optimizer, '\n\n')

In [ ]:
print('model.fc:', model.fc, '\n\n')

In [10]:
print('criterion:', criterion, '\n\n')
print('class_to_idx:', model.class_to_idx, '\n\n')
print('norm_mean:', norm_mean, '\n\n')
print('norm_std:', norm_std, '\n\n')

criterion: NLLLoss() 


class_to_idx: {'1': 0, '10': 1, '100': 2, '101': 3, '102': 4, '11': 5, '12': 6, '13': 7, '14': 8, '15': 9, '16': 10, '17': 11, '18': 12, '19': 13, '2': 14, '20': 15, '21': 16, '22': 17, '23': 18, '24': 19, '25': 20, '26': 21, '27': 22, '28': 23, '29': 24, '3': 25, '30': 26, '31': 27, '32': 28, '33': 29, '34': 30, '35': 31, '36': 32, '37': 33, '38': 34, '39': 35, '4': 36, '40': 37, '41': 38, '42': 39, '43': 40, '44': 41, '45': 42, '46': 43, '47': 44, '48': 45, '49': 46, '5': 47, '50': 48, '51': 49, '52': 50, '53': 51, '54': 52, '55': 53, '56': 54, '57': 55, '58': 56, '59': 57, '6': 58, '60': 59, '61': 60, '62': 61, '63': 62, '64': 63, '65': 64, '66': 65, '67': 66, '68': 67, '69': 68, '7': 69, '70': 70, '71': 71, '72': 72, '73': 73, '74': 74, '75': 75, '76': 76, '77': 77, '78': 78, '79': 79, '8': 80, '80': 81, '81': 82, '82': 83, '83': 84, '84': 85, '85': 86, '86': 87, '87': 88, '88': 89, '89': 90, '9': 91, '90': 92, '91': 93, '92': 94, '93': 95, '94': 96, '95': 9

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [4]:
def process_image(image_path):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns a Numpy array
    '''
    
    # DONE: Process a PIL image for use in a PyTorch model
    # Open the image
    img = PIL.Image.open(image_path)
    
    # Resize
    if img.size[0] > img.size[1]:
        img.resize((500, 256))
    else:
        img.resize((256, 500))
    
    # Crop
    crop_box = [(img.width - 224)/2, (img.height - 224)/2, (img.width - 224)/2 + 224, (img.height -224)/2 + 224]
    img = img.crop(crop_box)
    
    # Normalize the image
    img = np.array(img)/255
    img = (img - np.array(norm_mean)) / np.array(norm_std)
    
    # Move the color channel to the first dimension of the tensor
    img = img.transpose((2, 0, 1))
    
    return img

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [5]:
# Function to display an image from a pytorch tensor
def imshow(img, title=None):
    """Imshow for Tensor."""
    
    fig, ax = plt.subplots()
   
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    img = img.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing, normalize by default
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    img = np.clip(img, 0, 1)
    
    ax.imshow(img)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.tick_params(axis='both', length=0)
    ax.set_title(title)
    ax.set_xticklabels('')
    ax.set_yticklabels('')
    
    return ax

In [ ]:
# This uses testloader as an iterator and matches images from the folder to the correct
# keys and therefore the correct labels/names
images, labels = next(iter(testloader))

class_idx = {val: key for key, val in test_data.class_to_idx.items()}

# the name is from the key, not the string in the key.
# That is, labels will be from key 84 which refers to string '83'. That is folder 83 -> name -> hibiscus
name = cat_to_name[class_idx[labels[0].tolist()]]
imshow(images[0], name);

In [ ]:
image_path = 'C:/Users/jaket/Desktop/Udacity/Machine Learning with Python/aipnd-project-master/flowers/test/83/image_01789.jpg'
key = '83'

img = process_image(image_path)

image_tensor = torch.from_numpy(img).type(torch.FloatTensor)
img = image_tensor

name = cat_to_name[key]


imshow(img, name);

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [13]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model.to('cuda')
    
    # TODO: Implement the code to predict the class from an image file
    # Process the image so it is a NumPy array and no longer a PyTorch Tensor
    img = process_image(image_path)
    
    img_tensor = torch.from_numpy(img).type(torch.FloatTensor)
    
    model_input = img_tensor.unsqueeze(0).cuda()
    
    # Calculate the probabilities with softmax
    with torch.no_grad():
        output = model.forward(model_input)
        ps = torch.exp(output)
        top_probs, top_labels = ps.topk(topk)
        top_probs = top_probs.cpu().numpy().tolist()[0]
        top_labels = top_labels.cpu().numpy().tolist()[0]
    
    idx_to_class = {val: key for key, val in model.class_to_idx.items()}
    
    top_probs = [probs for probs in top_probs]
    top_labels = [idx_to_class[labels] for labels in top_labels]
    top_flowers = [cat_to_name[labels] for labels in top_labels]
    
    return top_probs, top_labels, top_flowers

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [14]:
# TODO: Display an image along with the top 5 classes

# Define the image path
print('testdir: flowers/test/14/image_06052.jpg')
image_path = 'flowers/test/14/image_06052.jpg'

# process the provided image
img = process_image(image_path)

# Find the key from the image_path
for directory in image_path.split('/'):
    if directory.isnumeric() == True:
        key = directory

# Numpy -> Tensor
image_tensor = torch.from_numpy(img).type(torch.FloatTensor)
img = image_tensor

# Find the image title
# The image title will need to be found using the cat_to_name.json. This will be merely a task of indexing.
title = cat_to_name[key]

prediction_results = predict(image_path, model)
top_probs, top_labels, top_flowers = prediction_results
print('Top probs:', top_probs)
print('Top labels:', top_labels)
print('Top flowers:', top_flowers)

# Deifne the bar graph in the second column. Only show the top 5 classes
sb.barplot(x = top_probs, y = top_flowers, color = sb.color_palette('tab10')[9])
imshow(img, title); # display the image in the second plot

testdir: flowers/test/14/image_06052.jpg


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.